In [ ]:
import pandas as pd

In [96]:
df = pd.read_csv('jeju.csv', encoding = 'cp949')
df.head(1)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20


In [ ]:
# 1. 주소 정보를 이용해서 위도, 경도 값을 저장합니다. '위치' 컬럼에 (위도, 경도) 저장
# 주소를 찾아서 저장하는 함수를 작성하여 처리
# 함수 이름은 geo_coder(query) 리턴값 : (위도, 경도) 만일 못찾았다면 '결과없음'으로 리턴
# 함수사용, apply()를 이용해 데이터 프레임에 적용

query = '제주특별자치도 제주시 중앙로7길 15(일도일동)'

def geo_coder(query):
    import requests
    REST_API_KEY = 'd44385bc69a339a727818cdd517beea9'
    query = query.split('(')[0]
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?category_group_code=PO3&query={query}"
    headers = {'Authorization':f'KakaoAK {REST_API_KEY}'}
    res = requests.get(url,headers=headers)
    jsondata = res.json()
    # print(query)    
    # print(jsondata)
    try:
        return jsondata['documents'][0]['y'],jsondata['documents'][0]['x']
    except:
        return '결과없음'

In [ ]:
df['위치'] = df['주소'].apply(geo_coder)
df.head(10)

In [ ]:
# 2. '위치' 컬럼의 값을 Marker 위치값으로, tooltip으로 읍면동을 사용하여 지도에 표시(클러스터 이용)
# 함수사용, apply()를 이용해 데이터 프레임에 적용

import folium
from folium.plugins import MarkerCluster

def display(loc,map):
    # print(loc[0],loc[1])
    if loc[0] != '결과없음':
        folium.Marker(loc[0], tooltip = loc[1]).add_to(map)
    

map = folium.Map((33.3616666,126.5291666), zoom_start = 8)
map_c = MarkerCluster().add_to(map)
df[['위치','읍면동']].apply(display, map = map_c, axis = 1)
map

In [ ]:
# 2-1. iterrows 활용

map = folium.Map((33.3616666,126.5291666), zoom_start = 8)
for index, row in df.iterrows():
    # print(index)
    # print(row['읍면동'],row['위치'])
    if row['위치'] != '결과없음':
        folium.Marker(row['위치'], tooltip = ['읍면동']).add_to(map)
map

In [ ]:
# 3. '일반현황' 컬럼의 인구수만 분리하여 '인구수'라는 int형 컬럼을 생성

df['일반현황'].str.split('/').str.get(0).str.split('(').str.get(0).str.split().str.get(-1).str.replace('명','').astype(int)

In [101]:
import re

df['인구수'] = df['일반현황'].apply(lambda x:int(re.findall('[0-9]{4,}',x)[-1]))
df.head(2)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,인구수
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,3146
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28 (일도이동),인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,38400
